# Exploration: remove spikes

Our goal is to eliminate spikes, and a good starting point is the Visvalingam–Whyatt algorithm.

This algorithm simplifies a curve by analyzing the triangular area formed by each point and its neighboring vertices.
Essentially, it just considers the previous and next vertices for each point.

According to Wikipedia, one disadvantage of the algorithm is that it 
"does not differentiate between sharp spikes and shallow features, meaning that it will clean up sharp spikes that may be important."
This limitation aligns with our goal, so we might consider a variation of the algorithm that also accounts for properties like angle and distance.

In [1]:
import geopandas as gpd

gdf = gpd.read_file("spiky-polygons.gpkg")
gdf.explore()

In [2]:
polygon = gdf.iloc[0].geometry
type(polygon)

shapely.geometry.polygon.Polygon

Identifying Key Properties

We'll create a GeoDataFrame containing each point along with its preceding and succeeding points.
The spike point corresponds to the eleventh entry in the data.

In [3]:
from shapely import Point

points = (Point(x) for x in polygon.exterior.coords)

points_gdf = gpd.GeoDataFrame(points, columns=["point"])
points_gdf = points_gdf.drop(points_gdf.index[-1])

points_gdf["previous"] = points_gdf["point"].shift(1)
points_gdf.at[0, "previous"] = points_gdf["point"].iloc[-1]

points_gdf["next"] = points_gdf["point"].shift(-1)
points_gdf.at[points_gdf.index[-1], "next"] = points_gdf["point"].iloc[0]

points_gdf

,point,previous,next
0,POINT (18.57016026617573 -33.912171286755274),POINT (8.293412877884293 -29.78021620019185),POINT (18.65947366303845 -33.844666975172984)
1,POINT (18.65947366303845 -33.844666975172984),POINT (18.57016026617573 -33.912171286755274),POINT (18.76488424189387 -33.85297519813696)
2,POINT (18.76488424189387 -33.85297519813696),POINT (18.65947366303845 -33.844666975172984),POINT (18.8038290370375 -33.93346110810046)
3,POINT (18.8038290370375 -33.93346110810046),POINT (18.76488424189387 -33.85297519813696),POINT (18.792405230462034 -34.0004461557475)
4,POINT (18.792405230462034 -34.0004461557475),POINT (18.8038290370375 -33.93346110810046),POINT (18.73009355823223 -34.02796714431566)
5,POINT (18.73009355823223 -34.02796714431566),POINT (18.792405230462034 -34.0004461557475),POINT (18.657915871232706 -34.03315978366815)
6,POINT (18.657915871232706 -34.03315978366815),POINT (18.73009355823223 -34.02796714431566),POINT (18.60858579738411 -34.004600267229485)
7,POINT (18.60858579738411 -34.004600267229485),POINT (18.657915871232706 -34.03315978366815),POINT (18.5795070170102 -33.963578416344866)
8,POINT (18.5795070170102 -33.963578416344866),POINT (18.60858579738411 -34.004600267229485),POINT (18.571198794046225 -33.91840245397825)
9,POINT (18.571198794046225 -33.91840245397825),POINT (18.5795070170102 -33.963578416344866),POINT (18.570244036171502 -33.91230820898571)


Now, let's perform some calculations.

In [4]:
from shapely import Polygon
import numpy as np


# Area
def area(x):
    return Polygon((x.previous, x.point, x.next)).area


points_gdf["area"] = points_gdf.apply(area, axis=1)


# Angle
def angle(x):
    p1 = x.previous
    p2 = x.point
    p3 = x.next

    # Convert Shapely points to numpy arrays
    a = np.array([p1.x, p1.y])
    b = np.array([p2.x, p2.y])
    c = np.array([p3.x, p3.y])

    # Create vectors
    ba = a - b
    bc = c - b

    # Calculate the cosine of the angle using dot product and magnitudes
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

    # Ensure the value is within the valid range for arccos
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

    # Calculate the angle in radians and then convert to degrees
    angle_radians = np.arccos(cosine_angle)
    angle_degrees = np.degrees(angle_radians)

    return angle_degrees


points_gdf["angle"] = points_gdf.apply(angle, axis=1)


# Distances
def distances(x):
    return (x.point.distance(x.previous), x.point.distance(x.next))


points_gdf["distances"] = points_gdf.apply(distances, axis=1)
points_gdf


,point,previous,next,area,angle,distances
0,POINT (18.57016026617573 -33.912171286755274),POINT (8.293412877884293 -29.78021620019185),POINT (18.65947366303845 -33.844666975172984),0.531382,121.014004,"(11.076307585117535, 0.11195407514403768)"
1,POINT (18.65947366303845 -33.844666975172984),POINT (18.57016026617573 -33.912171286755274),POINT (18.76488424189387 -33.85297519813696),0.003929,138.410896,"(0.11195407514403768, 0.10573748958365603)"
2,POINT (18.76488424189387 -33.85297519813696),POINT (18.65947366303845 -33.844666975172984),POINT (18.8038290370375 -33.93346110810046),0.004080,120.327599,"(0.10573748958365603, 0.08941296757982996)"
3,POINT (18.8038290370375 -33.93346110810046),POINT (18.76488424189387 -33.85297519813696),POINT (18.792405230462034 -34.0004461557475),0.001764,144.500748,"(0.08941296757982996, 0.06795218881647218)"
4,POINT (18.792405230462034 -34.0004461557475),POINT (18.8038290370375 -33.93346110810046),POINT (18.73009355823223 -34.02796714431566),0.001930,123.507710,"(0.06795218881647218, 0.06811864141219681)"
5,POINT (18.73009355823223 -34.02796714431566),POINT (18.792405230462034 -34.0004461557475),POINT (18.657915871232706 -34.03315978366815),0.000831,160.285460,"(0.06811864141219681, 0.07236423152391125)"
6,POINT (18.657915871232706 -34.03315978366815),POINT (18.73009355823223 -34.02796714431566),POINT (18.60858579738411 -34.004600267229485),0.001159,145.816508,"(0.07236423152391125, 0.057000896178203364)"
7,POINT (18.60858579738411 -34.004600267229485),POINT (18.657915871232706 -34.03315978366815),POINT (18.5795070170102 -33.963578416344866),0.000597,155.399914,"(0.057000896178203364, 0.050282876986443875)"
8,POINT (18.5795070170102 -33.963578416344866),POINT (18.60858579738411 -34.004600267229485),POINT (18.571198794046225 -33.91840245397825),0.000486,155.089381,"(0.050282876986443875, 0.04593358405969221)"
9,POINT (18.571198794046225 -33.91840245397825),POINT (18.5795070170102 -33.963578416344866),POINT (18.570244036171502 -33.91230820898571),0.000004,178.483184,"(0.04593358405969221, 0.006168580438677504)"


The area parameter doesn't seem to be very effective, but the angle parameter is quite reliable.
Distance might be useful if we're specifically interested in long spikes, like the ones in the example.
However, we could argue that regardless of distance, points with a very small angle could still be considered spikes.